In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [2]:
class Model1 (nn.Module):
    def __init__(self):
        super(Model1,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(5,20),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Linear(20,10),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Linear(10,1),
            nn.Tanh()
        )
    def forward(self,x):
        return self.model(x)

In [3]:
df = pd.read_json("data3/normal_vpm.json",orient="records",lines=True)


In [4]:
list_data = df.to_numpy()
np.random.shuffle(list_data)
test_size = 0.1
test_split = int((1-test_size)*list_data.size)

data_x = torch.Tensor(list_data[:,:-1])
data_Y = torch.Tensor(list_data[:,-1].reshape(-1,1))
data = TensorDataset(data_x,data_Y)
train_data, test_dataset = torch.utils.data.random_split(data,[18000,2348])

train_dataLoader = DataLoader(train_data,batch_size=128,shuffle=True)
test_dataLoader = DataLoader(test_dataset,batch_size=128,shuffle=True)


In [5]:
model1 = Model1()
optimizer = optim.Adam(model1.parameters(),lr=0.01)
criterion = nn.MSELoss()

In [6]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model1.to(device)
print("Running model on:",device)

Running model on: cpu


In [7]:
epochs = 50

model1.train()
for epoch in tqdm(range(epochs)):
    train_loss = []
    for x, Y in train_dataLoader:
        x = x.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        out = model1(x)
        loss = criterion(out, Y)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    print(f"Loss: {np.average(train_loss):.3f}")

  4%|▍         | 2/50 [00:00<00:07,  6.23it/s]Loss: 0.401
Loss: 0.276
  8%|▊         | 4/50 [00:00<00:07,  6.44it/s]Loss: 0.249
Loss: 0.214
 12%|█▏        | 6/50 [00:00<00:06,  6.46it/s]Loss: 0.205
Loss: 0.194
 16%|█▌        | 8/50 [00:01<00:06,  6.55it/s]Loss: 0.193
Loss: 0.187
 20%|██        | 10/50 [00:01<00:06,  6.60it/s]Loss: 0.184
Loss: 0.187
 24%|██▍       | 12/50 [00:01<00:05,  6.78it/s]Loss: 0.180
Loss: 0.183
 28%|██▊       | 14/50 [00:02<00:05,  6.81it/s]Loss: 0.181
Loss: 0.177
 32%|███▏      | 16/50 [00:02<00:05,  6.74it/s]Loss: 0.173
Loss: 0.168
 36%|███▌      | 18/50 [00:02<00:04,  6.74it/s]Loss: 0.166
Loss: 0.171
 40%|████      | 20/50 [00:03<00:04,  6.64it/s]Loss: 0.166
Loss: 0.169
 44%|████▍     | 22/50 [00:03<00:04,  6.90it/s]Loss: 0.171
Loss: 0.167
 48%|████▊     | 24/50 [00:03<00:03,  6.79it/s]Loss: 0.168
Loss: 0.167
 52%|█████▏    | 26/50 [00:03<00:03,  6.68it/s]Loss: 0.171
Loss: 0.166
 56%|█████▌    | 28/50 [00:04<00:03,  6.64it/s]Loss: 0.170
Loss: 0.168
 60%|█████

In [8]:
correct = 0
errors = []
for x, Y in test_dataLoader:
    model1.eval()
    x = x.to(device)
    Y = Y.cpu()
    out=model1(x)
    out.cpu()
    for i,o in enumerate(out):
        if o * Y[i] > 0:
            correct += 1
        else:
            errors.append([x[i],Y[i]])
print(f'Corectly predicted: {correct} out of {len(test_dataset)}\nTest accuracy: {correct/len(test_dataset):.2%}')


Corectly predicted: 2201 out of 2348
Test accuracy: 93.74%


Przykładowa wartość dla której nie ma zakupu, zobaczymy czy dzięki naszemu systemowi byli byśmy wstanie przekonać do zakupu
{"d_time":0.100068132,"n_visits":2,"visits_per_minute":0.0001321401,"max_disc":0,"curr_disc":0,"is_purchased":-1}

In [9]:
model1.cpu()
print(model1(torch.Tensor(np.array([[0.100068132,2,0.0001321401,0,0],[0.100068132,2,0.0001321401,0,5],[0.100068132,2,0.0001321401,0,10],[0.100068132,2,0.0001321401,0,15],[0.100068132,2,0.0001321401,0,20]]))))
#print(model1(torch.Tensor(np.array([60000000,2,0,0]))))

tensor([[-0.9995],
        [-0.9969],
        [-0.9817],
        [-0.8782],
        [-0.6020]], grad_fn=<TanhBackward>)


In [10]:
errors

[[tensor([4.0642e-01, 4.0000e+00, 6.5072e-05, 1.0000e+01, 1.0000e+01]),
  tensor([1.])],
 [tensor([6.1584e-01, 6.0000e+00, 6.4415e-05, 2.0000e+01, 0.0000e+00]),
  tensor([1.])],
 [tensor([5.8953e-01, 5.0000e+00, 5.6074e-05, 0.0000e+00, 0.0000e+00]),
  tensor([1.])],
 [tensor([6.0037e-01, 3.0000e+00, 3.3037e-05, 0.0000e+00, 0.0000e+00]),
  tensor([1.])],
 [tensor([2.9492e-01, 5.0000e+00, 1.1209e-04, 1.5000e+01, 5.0000e+00]),
  tensor([1.])],
 [tensor([2.9944e-01, 3.0000e+00, 6.6238e-05, 2.0000e+01, 2.0000e+01]),
  tensor([1.])],
 [tensor([3.2685e-01, 5.0000e+00, 1.0114e-04, 1.5000e+01, 1.0000e+01]),
  tensor([1.])],
 [tensor([3.4255e-01, 7.0000e+00, 1.3511e-04, 2.0000e+01, 0.0000e+00]),
  tensor([1.])],
 [tensor([3.3132e-01, 3.0000e+00, 5.9866e-05, 0.0000e+00, 0.0000e+00]),
  tensor([1.])],
 [tensor([3.4629e-01, 6.0000e+00, 1.1455e-04, 1.5000e+01, 1.0000e+01]),
  tensor([1.])],
 [tensor([8.8581e-02, 4.0000e+00, 2.9855e-04, 0.0000e+00, 0.0000e+00]),
  tensor([-1.])],
 [tensor([2.7088e-01